In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
num_var = 50      # number of variables
num_ineq = 50     # number of constraints
num_data = 10000  # number of data
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# data sample from uniform distribution
b_samples = torch.from_numpy(np.random.uniform(-1, 1, size=(num_data, num_ineq))).float()
data = {"b":b_samples}
# data split
from src.utlis import data_split
data_train, data_test, data_dev = data_split(data, test_size=test_size, val_size=val_size)

In [5]:
# torch dataloaders
from torch.utils.data import DataLoader
batch_size = 64
loader_train = DataLoader(data_train, batch_size, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=False)

## Exact Solver

In [6]:
from src.problem import msLinear
model = msLinear(num_var, num_ineq, timelimit=60)

In [7]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # set params
    model.set_param_val({"b":b.cpu().numpy()})
    # solve
    tick = time.time()
    xval, objval = model.solve("gurobi")
    tock = time.time()
    # eval
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    if sum(model.cal_violation()) > 0:
        xval, objval = model.solve("gurobi", tee=True)
        break
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/ln_exact_50-50.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 100/100 [1:42:06<00:00, 61.27s/it]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000             100.0    100.000000
mean   -56.341032               0.0     61.249173
std      0.414810               0.0      0.143941
min    -57.277715               0.0     60.988047
25%    -56.660185               0.0     61.158013
50%    -56.374159               0.0     61.217180
75%    -56.048647               0.0     61.323208
max    -55.268315               0.0     61.749841
Number of infeasible solution: 0


## Heuristic - Round

In [8]:
from src.heuristic import naive_round

In [9]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # set params
    model.set_param_val({"b":b.cpu().numpy()})
    # relax
    model_rel = model.relax()
    # solve
    tick = time.time()
    xval_rel, _ = model_rel.solve("gurobi")
    xval, objval = naive_round(xval_rel, model)
    tock = time.time()
    # eval
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/ln_heur_rnd_50-50.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:22<00:00,  4.42it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean   -56.505190          1.198322      0.146128
std      0.430429          0.378845      0.009731
min    -57.489181          0.245157      0.126043
25%    -56.814062          0.947998      0.140068
50%    -56.511283          1.137925      0.144286
75%    -56.219426          1.431795      0.150272
max    -55.470035          2.188033      0.176831
Number of infeasible solution: 100


## Heuristic - N1

In [10]:
model_heur = model.first_solution_heuristic(nodes_limit=1)

In [11]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # set params
    model_heur.set_param_val({"b":b.cpu().numpy()})
    # solve
    tick = time.time()
    xval, objval = model_heur.solve("gurobi")
    tock = time.time()
    # eval
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model_heur.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/ln_heur_n1_50-50.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:16<00:00,  6.20it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000             100.0    100.000000
mean   -35.219012               0.0      0.151665
std     11.121363               0.0      0.006707
min    -55.455964               0.0      0.133031
25%    -39.097745               0.0      0.147781
50%    -35.513947               0.0      0.149934
75%    -29.622997               0.0      0.155645
max     -4.621126               0.0      0.170249
Number of infeasible solution: 0


## Learnable Rounding

In [12]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [13]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 128           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [14]:
# set problem
import neuromancer as nm
from src.problem import nmLinear
from src.func.layer import netFC
from src.func import roundGumbelModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmLinear(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [15]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs=epochs, patience=patience, warmup=warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 1485.61
Epoch 1, Validation Loss: -25.41
Epoch 2, Validation Loss: -38.08
Epoch 3, Validation Loss: -39.81
Epoch 4, Validation Loss: -35.75
Epoch 5, Validation Loss: -41.87
Epoch 6, Validation Loss: -42.21
Epoch 7, Validation Loss: -8.57
Epoch 8, Validation Loss: -38.46
Epoch 9, Validation Loss: 41.41
Epoch 10, Validation Loss: -43.10
Epoch 11, Validation Loss: -44.34
Epoch 12, Validation Loss: -25.56
Epoch 13, Validation Loss: 7.98
Epoch 14, Validation Loss: 3.51
Epoch 15, Validation Loss: -42.16
Epoch 16, Validation Loss: -36.62
Epoch 17, Validation Loss: 93.65
Epoch 18, Validation Loss: -40.55
Epoch 19, Validation Loss: -43.61
Epoch 20, Validation Loss: -44.07
Epoch 21, Validation Loss: -46.37
Epoch 22, Validation Loss: -41.72
Epoch 23, Validation Loss: -47.76
Epoch 24, Validation Loss: -38.95
Epoch 25, Validation Loss: -48.81
Epoch 26, Validation Loss: -47.69
Epoch 27, Validation Loss: -28.43
Epoch 28, Validation Loss: -49.06
Epoch 29, Validation Loss: -48

In [16]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/ln_lr_50-50.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 62.18it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean   -52.914468          0.000330      0.003360
std      0.612080          0.003300      0.000911
min    -53.245679          0.000000      0.001348
25%    -53.162749          0.000000      0.002999
50%    -53.119023          0.000000      0.003315
75%    -53.006839          0.000000      0.003742
max    -49.775469          0.033004      0.007266
Number of infeasible solution: 1


## Learnable Threshold

In [17]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [18]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 128           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [19]:
# set problem
import neuromancer as nm
from src.problem import nmLinear
from src.func.layer import netFC
from src.func import roundThresholdModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmLinear(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [20]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs=epochs, patience=patience, warmup=warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 1241.19
Epoch 1, Validation Loss: -28.89
Epoch 2, Validation Loss: -18.77
Epoch 3, Validation Loss: -19.46
Epoch 4, Validation Loss: -37.97
Epoch 5, Validation Loss: -13.81
Epoch 6, Validation Loss: -22.86
Epoch 7, Validation Loss: -39.14
Epoch 8, Validation Loss: -20.54
Epoch 9, Validation Loss: -21.41
Epoch 10, Validation Loss: -33.46
Epoch 11, Validation Loss: -44.99
Epoch 12, Validation Loss: -45.39
Epoch 13, Validation Loss: -43.75
Epoch 14, Validation Loss: -41.14
Epoch 15, Validation Loss: -22.29
Epoch 16, Validation Loss: 25.85
Epoch 17, Validation Loss: -44.66
Epoch 18, Validation Loss: -45.66
Epoch 19, Validation Loss: -44.17
Epoch 20, Validation Loss: 30.63
Epoch 21, Validation Loss: -47.11
Epoch 22, Validation Loss: -47.81
Epoch 23, Validation Loss: -42.74
Epoch 24, Validation Loss: -14.68
Epoch 25, Validation Loss: -47.94
Epoch 26, Validation Loss: -40.11
Epoch 27, Validation Loss: -44.99
Epoch 28, Validation Loss: -48.32
Epoch 29, Validation Loss

In [21]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/ln_lt_50-50.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 71.40it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000             100.0    100.000000
mean   -52.785579               0.0      0.003139
std      0.138392               0.0      0.000993
min    -53.229900               0.0      0.001863
25%    -52.826061               0.0      0.002485
50%    -52.773664               0.0      0.003001
75%    -52.708682               0.0      0.003510
max    -52.537535               0.0      0.007511
Number of infeasible solution: 0


### Parametric Learning Then Rounding

In [22]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [23]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hsize = 128           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [24]:
# set problem
import neuromancer as nm
from src.problem import nmLinear
from src.func.layer import netFC
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# build neuromancer problem for rounding
components = nn.ModuleList([smap]).to("cuda")
loss_fn = nmLinear(["b", "x"], num_var, num_ineq, penalty_weight)

In [25]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs=epochs, patience=patience, warmup=warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 1248.58
Epoch 1, Validation Loss: -28.31
Epoch 2, Validation Loss: -37.70
Epoch 3, Validation Loss: -39.67
Epoch 4, Validation Loss: 61.90
Epoch 5, Validation Loss: 26.19
Epoch 6, Validation Loss: -40.54
Epoch 7, Validation Loss: -41.64
Epoch 8, Validation Loss: -43.38
Epoch 9, Validation Loss: -42.46
Epoch 10, Validation Loss: -34.45
Epoch 11, Validation Loss: -35.01
Epoch 12, Validation Loss: -44.19
Epoch 13, Validation Loss: -42.28
Epoch 14, Validation Loss: -44.92
Epoch 15, Validation Loss: -38.01
Epoch 16, Validation Loss: -42.01
Epoch 17, Validation Loss: -43.87
Epoch 18, Validation Loss: -45.46
Epoch 19, Validation Loss: -46.85
Epoch 20, Validation Loss: -46.36
Epoch 21, Validation Loss: -45.84
Epoch 22, Validation Loss: -42.75
Epoch 23, Validation Loss: -46.10
Epoch 24, Validation Loss: -44.20
Epoch 25, Validation Loss: -40.65
Epoch 26, Validation Loss: -48.22
Epoch 27, Validation Loss: -46.70
Epoch 28, Validation Loss: -48.23
Epoch 29, Validation Loss

In [26]:
from src.heuristic import naive_round
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval_rel, _ = model.get_val()
    xval, objval = naive_round(xval_rel, model)
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/ln_pr_50-50.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 65.30it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean   -52.839183          0.005981      0.000910
std      0.113815          0.027682      0.000534
min    -52.945929          0.000000      0.000000
25%    -52.945929          0.000000      0.000997
50%    -52.862097          0.000000      0.001000
75%    -52.730474          0.000000      0.001009
max    -52.553603          0.180119      0.003002
Number of infeasible solution: 6
